In [5]:
#Import des biblothèques utiles

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.linalg as la
from time import process_time

## Présentation de la méthode des éléments finis pour TDSE


We will apply the Crank-Nicolson (CN) scheme to integrate numerically Eq. 1 in one space dimension with Neumann boundary conditions.

$$i \frac{\partial \psi}{\partial t} = -\frac{1}{2} \frac{\partial^2 \psi}{\partial x^2} + V(x) \psi - x E(t) \psi, \qquad (Eq. 1)$$



# Finite Difference Methods

The above Schrödinger equation describes the time evolution of the variable $\psi(x,t)$ in one space dimension.

To approximate $\psi(x,t)$ numerically, we discretize the coordinate system with a two-dimensional regular grid.

Let us discretize both time and space:
$$t_n = n \Delta t, \quad n = 0, \dots, N - 1,$$
$$x_j = j \Delta x, \quad j = 0, \dots, J - 1,$$

where $N$ and $J$ are the number of discrete time and space points, respectively, and $Δt$ and $Δx$ are the time and space steps:
$$Δt = T / N, \quad Δx = L / J,$$

where $T$ is the final integration time.

Our goal is to approximate the unknown analytic solution $\psi(x,t)$ at discrete grid points, using the notation:
$$\Psi^n_j = \psi(jΔx, nΔt) \approx \psi(jΔx, nΔt).$$

# The Crank-Nicolson Stencil

To approximate the time derivative at grid point $(j,n)$, we use:
$$i \left. \frac{\partial \psi}{\partial t} \right|_{x = jΔx, t = nΔt} \approx \frac{\Psi^{n+1}_j - \Psi^n_j}{Δt}.$$

The spatial part of the Crank-Nicolson stencil for the Schrödinger equation ($i \psi_t = -\frac{1}{2} \psi_{xx} + V(x) \psi - x E(t) \psi$) approximates the Laplacian and potential operators as follows:
$$-\frac{1}{2} \left. \frac{\partial^2 \psi}{\partial x^2} \right|_{x = jΔx, t = nΔt} \approx \frac{-1}{4Δx^2} \left( \Psi^n_{j+1} - 2 \Psi^n_j + \Psi^n_{j-1} + \Psi^{n+1}_{j+1} - 2 \Psi^{n+1}_j + \Psi^{n+1}_{j-1} \right),$$
$$V(x_j) \psi(jΔx, nΔt) \approx \frac{V(x_j)}{2} (\Psi^n_j + \Psi^{n+1}_j),$$
$$x_j E(nΔt) \psi(jΔx, nΔt) \approx \frac{x_j E(nΔt)}{2} (\Psi^n_j + \Psi^{n+1}_j).$$

Applying this stencil to the grid point $(j,n)$ gives:
$$i\frac{\Psi^{n+1}_j - \Psi^n_j}{Δt} = \frac{-1}{4Δx^2} (\Psi^n_{j+1} - 2 \Psi^n_j + \Psi^n_{j-1} + \Psi^{n+1}_{j+1} - 2 \Psi^{n+1}_j + \Psi^{n+1}_{j-1}) + \frac{V(x_j)}{2} (\Psi^n_j + \Psi^{n+1}_j) - \frac{x_j E(nΔt)}{2} (\Psi^n_j + \Psi^{n+1}_j).$$



In [3]:
class CrankNicolson:
    
    def set_grid(self, x_min, x_max, n_x, t_min, t_max, n_t):

        self.x_min, self.x_max, self.n_x = x_min, x_max, n_x
        self.t_min, self.t_max, self.n_t = t_min, t_max, n_t
        self.x_pts, self.delta_x = np.linspace(x_min, x_max, n_x, retstep=True, endpoint=False)
        self.t_pts, self.delta_t = np.linspace(t_min, t_max, n_t, retstep=True, endpoint=False)
        
    def set_parameter(self,f):
        
        self.f = f
    
    def _make_tridiag(self, sig, n, data_type):
        
        M = np.diagflat(np.full(n, (1_2*sig), dtype=data_type)) +\
            np.diagflat(np.full(n-1, (-sig), dtype=data_type), 1) +\
            np.diagflat(np.full(n-1, (-sig), dtype=data_type), -1)
        
        return M
    
    def solve(self, psi_init):
        
        sig = (1j*self.delta_t)/(4*(self.delta_x)**2)
        data_type = type(psi_init[0]*sig)
        
        self.psi_matrix = np.zeros([self.n_t, self.n_x],dtype=data_type)
        
        A = self._make_tridiag(self,sig, self.n_x,data_type)
        B = self._make_tridiag(self, -sig, self.n_x, data_type)
        
        for i in [0,1]:
            A[-i,-i] = 1.0
            A[i,1-3*i] = 0.0
            B[-b,-b] = 0.0
            B[-b,1-3*b] = 0.0
        
        psi = psi_init
        
        for k in range(self.n_t):
            self.psi_matrix[k,:] = psi
            fpsi = self.f(psi)
            if n==0: fpsi_old = fpsi
            psi = la.solve(A, B.dot(psi) + self.delta_t * (1.5 * fpsi - 0.5 * fpsi_old))
            fpsi_old = fpsi
        
    def get_final_psi(self):
        return self.psi_matrix[-1,:].copy()
    
           
            
        
        